# Prediction

Attempting to finetune Vgg16 model to solve https://www.kaggle.com/c/state-farm-distracted-driver-detection.

This notebook uses techniques described by the competition winner: https://www.kaggle.com/c/state-farm-distracted-driver-detection/discussion/22906.

In [1]:
import importlib, sys
sys.path.insert(0, './../../utils')
import utils; importlib.reload(utils)
from utils import *

%matplotlib inline

K.set_image_dim_ordering('th')
# base_dir = '../sample/'
base_dir = '../'
batch_size = 64
output_size = 10
dropout = 0.8
size = (224,224)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [3]:
gen = image.ImageDataGenerator()
gen_with_aug = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

batches = gen.flow_from_directory(base_dir + 'cropped_train/', target_size=size, class_mode='categorical', shuffle=False, batch_size=batch_size)
batches_aug = gen_with_aug.flow_from_directory(base_dir + 'cropped_train/', target_size=size, class_mode='categorical', shuffle=True, batch_size=batch_size)
val_batches = gen.flow_from_directory(base_dir + 'cropped_valid/', target_size=size, class_mode='categorical', shuffle=False, batch_size=batch_size)
test_batches = gen.flow_from_directory(base_dir + 'cropped_test/', target_size=size, class_mode='categorical', shuffle=False, batch_size=batch_size)

trn_labels = to_categorical(batches.classes)
val_labels = to_categorical(val_batches.classes)

Found 17850 images belonging to 10 classes.
Found 17850 images belonging to 10 classes.
Found 4574 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


## Convolutional pre-computing

In [3]:
# Train/validation set pre-computing to speed up fine tuning
m = vgg(output_size, dropout=dropout)
cm = conv_model(m)

batches.reset()
trn_features = cm.predict_generator(batches, steps(batches, batch_size), verbose=1)
save_array(base_dir + 'models/train_convlayer_features.bc', trn_features)

val_batches.reset()
val_features = cm.predict_generator(val_batches, steps(val_batches, batch_size), verbose=1)
save_array(base_dir + 'models/valid_convlayer_features.bc', val_features)

72/72 [==============================] - 110s   


In [ ]:
# Test set convolutional features to compute nearest neighbors
m = vgg(output_size, dropout=dropout)
cm = conv_model(m)
cm.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
test_batches.reset()
test_features = cm.predict_generator(test_batches, steps(test_batches, batch_size), verbose=1)
save_array(base_dir + 'models/test_convlayer_features.bc', test_features)

In [4]:
trn_features = load_array(base_dir + 'models/train_convlayer_features.bc')
val_features = load_array(base_dir + 'models/valid_convlayer_features.bc')

## Finetuning

In [2]:
def ft_last(model, index):
    dm = vgg_fc_model(model, output_size, dropout=dropout)
    for i in range(len(dm.layers)): dm.layers[i].trainable = i >= len(dm.layers) - 3
    
    params = [[RMSProp(1e-3), 20], [RMSProp(1e-4), 20]]
    fits_w_features(dm, params, trn_features, trn_labels, val_features, val_labels)
    
    for l1,l2 in zip(model.layers[last_conv_idx(model) + 1:], dm.layers): l1.set_weights(l2.get_weights())
    model.save_weights(base_dir + 'models/bb_last' + str(index) + '.h5')
    
def ft_dense(model, index):
    dm = vgg_fc_model(model, output_size, dropout=dropout)
    for l in dm.layers: l.trainable = True
    
    params = [[RMSProp(1e-4), 20], [RMSProp(1e-5), 20]]
    fits_w_features(dm, params, trn_features, trn_labels, val_features, val_labels)
    
    for l1,l2 in zip(model.layers[last_conv_idx(model) + 1:], dm.layers): l1.set_weights(l2.get_weights())
    model.save_weights(base_dir + 'models/bb_dense' + str(index) + '.h5')
    
def ft_aug(model, index):
    for i in range(len(model.layers)): model.layers[i].trainable = i >= 16
    params = [[RMSProp(1e-5), 10], [RMSProp(1e-6), 10]]
    for i in range(6): model_filename=base_dir + 'models/bb_aug' + str(index) + '.h5'
    fits_w_batches(model, params, batches_aug, val_batches, model_filename=model_filename)

## Training

In [ ]:
clear_logs()
model = vgg(output_size, dropout=dropout)
model.load_weights(base_dir + 'models/bb_dense0.h5')
# ft_last(model, 0)
# ft_dense(model, 0)
ft_aug(model, 0)
log_action("Finished training model")

In [ ]:
model = vgg(output_size, dropout=dropout)
model.load_weights(base_dir + 'models/dense_pseudo_aug0.h5')

## K-nearest neighbor

- Did not vary the number of neighbor (currently k = 10).
- Did optimize the weight of neighbors in final prediction

In [ ]:
test_features = load_array(base_dir + 'models/test_convlayer_features.bc')
nsamples, nfilters, nx, ny = test_features.shape
test_features = test_features.reshape((nsamples,nfilters*nx*ny))
n=len(test_features)
nb_neighbors = 10

step = 40000
result = []
for i in tqdm(range(0, n, step)):
    start_idx, end_idx = i, min(n, i+step)
    nn = NearestNeighbors(nb_neighbors + 1, metric='cosine', algorithm='brute').fit(test_features)
    dists, idxs = nn.kneighbors([test_features[j] for j in range(start_idx, end_idx)])
    result += [np.vstack((idxs[i],dists[i])).T[1:] for i in range(end_idx-start_idx)]
    
save_array(base_dir + 'models/10_neighbors2.bc', np.array(result))

## Visualize results

- Did not visualize false positives, false negatives, most certain, most uncertain, etc.

Main confusions seem to be:

- c8 taken for c7 -> 162
- c8 taken for c2 -> 83
- c9 taken for c7 -> 68
- c9 taken for c0 -> 44
- c8 taken for c6 -> 36
- c0 taken for c3 -> 33

In [ ]:
predicted_labels = np.array(all_preds[-1]).argmax(axis=1)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(val_batches.classes, predicted_labels)

plot_confusion_matrix(cm, val_batches.class_indices)

## Submit to Kaggle

- Did optimize clipping rate but did not play with asymetric clipping rates (different values for min and max).

In [ ]:
test_batches.reset()
preds = model.predict_generator(test_batches, steps(test_batches, batch_size), verbose=1)
save_array(base_dir + 'models/test_predictions.bc', preds)

In [ ]:
preds = load_array(base_dir + 'models/test_predictions.bc')
nearest_neighbors = load_array(base_dir + 'models/10_neighbors2.bc')[:]
weighted_preds = neighbor_averaged_preds(nearest_neighbors, preds)

mx = 0.93
clipped_preds = np.clip(weighted_preds, (1-mx)/9, mx)

classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(clipped_preds, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
file_path = base_dir + 'submission.gz'
submission.to_csv(file_path, index=False, compression='gzip')
    
from IPython.display import FileLink
FileLink(file_path)

## TO DO

- Generate cropped images or use preprocessing
- Train on cropped images
- Check score with aug + neighbor average

## Not tried

- detector: fully convolutional NN w/ bigger images
- detector: couple with category model to provide context
- detector: crop only chest rather than entire body (tip of head, left of head, right of steering wheel, waist)
- Categorical ensembling
- Segment average (if the majority of similar images are certain, normalize all similar images to the same prediction)
- Verify pseudo labeling efficiency on final model